In [7]:
import warnings
import itertools
import pandas
import math
import sys
import os
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [8]:
# Add detector summary file to check scores for a given profile (Note that if there are missing fields in the file)

detector_summary_file = "../results/expose/expose_standard_scores.csv"
#colomn name of the profile score in result files
detector_profile = "S(t)_standard"

In [9]:
#Reading summary
Error = None
if os.path.isfile(detector_summary_file):
    summaryDataFrame = pandas.read_csv(detector_summary_file,index_col="File")
    if 'Score' in summaryDataFrame.columns:
        summaryDataFrame = summaryDataFrame.sort_values(by=['Score'],ascending=False)
        print("Printing scores for selected profile (NaN represents total score):")
        print(summaryDataFrame.loc[:, 'Score'])
    else:
        Error = "'Score' colomn mission in data file"
        print(Error)
else:
    Error = "No such file : "+detector_summary_file
    print(Error)

Printing scores for selected profile (NaN represents total score):
File
realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv             1.499566
realTraffic/occupancy_t4013.csv                              1.379261
realTraffic/speed_t4013.csv                                  1.228860
artificialWithAnomaly/art_daily_jumpsdown.csv                0.790642
artificialWithAnomaly/art_daily_jumpsup.csv                  0.784011
realAWSCloudwatch/ec2_cpu_utilization_ac20cd.csv             0.767313
realKnownCause/ec2_request_latency_system_failure.csv        0.726442
artificialWithAnomaly/art_daily_flatmiddle.csv               0.476161
realAWSCloudwatch/ec2_cpu_utilization_825cc2.csv             0.392629
realTweets/Twitter_volume_CVS.csv                            0.309561
realAWSCloudwatch/grok_asg_anomaly.csv                       0.039000
realAWSCloudwatch/ec2_network_in_5abac7.csv                  0.018516
artificialNoAnomaly/art_daily_no_noise.csv                   0.000000
artificialNoAnomal

In [13]:
# select detector summary file row(data file name) from above
detector_summary_row = "realAWSCloudwatch/ec2_cpu_utilization_24ae8d.csv"

# Add detector result file(a single file for specific data file)
result_file = "../results/expose/realAWSCloudwatch/expose_ec2_cpu_utilization_24ae8d.csv"

In [14]:
if not summaryDataFrame.index.contains(detector_summary_row):
    Error = "No detector_summary_row '"+detector_summary_row+"' in file "+detector_summary_file
else:
    Error = None

if set(['Threshold','TP','TN','FP','FN','Total_Count']).issubset(summaryDataFrame.columns) and Error is None:
    if summaryDataFrame["Threshold"][detector_summary_row].size == 1:
        threshold = summaryDataFrame["Threshold"][detector_summary_row]
        TP = summaryDataFrame["TP"][detector_summary_row]
        TN = summaryDataFrame["TN"][detector_summary_row]
        FP = summaryDataFrame["FP"][detector_summary_row]
        FN = summaryDataFrame["FN"][detector_summary_row]
        total_Count = summaryDataFrame["Total_Count"][detector_summary_row]
    else:
        threshold = summaryDataFrame["Threshold"][detector_summary_row][0]
        TP = summaryDataFrame["TP"][detector_summary_row][0]
        TN = summaryDataFrame["TN"][detector_summary_row][0]
        FP = summaryDataFrame["FP"][detector_summary_row][0]
        FN = summaryDataFrame["FN"][detector_summary_row][0]
        total_Count = summaryDataFrame["Total_Count"][detector_summary_row][0]
else:
    if Error is None:
        Error = "Missing colomns in file "+detector_summary_file
    print(Error)
    print("Run from beginng to clear Error")

In [15]:
#Ploting Result (Note if the plot is not visible please close the tab shutdown notebook and restart)
#Reading results file 
if os.path.isfile(result_file):
    dataframe = pandas.read_csv(result_file)
    Error = None
else:
    Error = "No such file : "+result_file
    print(Error)

if set(['timestamp','value','anomaly_score','label', detector_profile]).issubset(dataframe.columns)\
        and Error is None:
    x = np.array(dataframe['timestamp'])
    value = np.array(dataframe['value'])
    anomaly_score = np.array(dataframe['anomaly_score'])
    anomaly_label = np.array(dataframe['label'])

    standard_score = np.array(dataframe[detector_profile])
else:
    if Error is None:
        Error = "Missing colomns in file "+result_file
    print(Error)
# Plot values, anomaly score and label scaled to values

if Error is None:
    value_max = np.max(value)
    trace_value = {"x": x,
             "y": value,
             "mode": 'lines',
             "name": 'Value'}

    trace_anomaly_score = {"x": x,
                  "y": anomaly_score*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly score'}

    trace_anomaly_label = {"x": x,
                  "y": anomaly_label*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly window'}
    trace_threshold = {"x": x,
                  "y": np.ones(len(x))*threshold*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly threshold'}

    traces = [trace_value,trace_anomaly_score,trace_threshold,trace_anomaly_label]
    layout = dict(title = "Scalled anomaly score with value : "+result_file,
                  xaxis = dict(title = 'X'),
                  yaxis = dict(title = 'Value')
                 )
    fig = dict(data=traces, layout=layout)
    iplot(fig)

    #plot, anomalys score, label, and result from benchmark
    trace_anomaly_score = {"x": x,
                  "y": anomaly_score,
                  "mode": 'lines',
                  "name": 'Anomaly score'}

    trace_anomaly_label = {"x": x,
                  "y": anomaly_label,
                  "mode": 'lines',
                  "name": 'Anomaly window'}

    trace_threshold = {"x": x,
                  "y": np.ones(len(x))*threshold,
                  "mode": 'lines',
                  "name": 'Anomaly threshold'}

    trace_standard_score = {"x": x,
                  "y": standard_score,
                  "mode": 'lines',
                  "name": 'standard_score'}

    traces = [trace_anomaly_score,trace_threshold,trace_anomaly_label,trace_standard_score]
    layout = dict(title = "Anomaly score : "+result_file,
                  xaxis = dict(title = 'X'),
                  yaxis = dict(title = 'Value')
                 )
    fig = dict(data=traces, layout=layout)
    iplot(fig)
else:
    print(Error)
    print("Run from beginng to clear Error")

In [16]:
if Error is None:
    TP,TN,FP,FN = 0,0,0,0
    for x in standard_score:
        if x > 0:
            TP +=1
        elif x == 0:
            TN +=1
        elif x == -0.11:
            FP +=1
        elif x == -1:
            FN +=1
    print("For result file : " + result_file)
    print("True Positive (Detected anomalies) : " + str(TP))
    print("True Negative (Detected non anomalies) : " + str(TN))
    print("False Positive (False alarms) : " + str(FP))
    print("False Negative (Anomaly not detected) : " + str(FN))
    print("Total data points : " + str(total_Count))
    print(detector_profile+" score : "+str(np.sum(standard_score)))
else:
    print(Error)
    print("Run from beginng to clear Error")

For result file : ../results/expose/realAWSCloudwatch/expose_ec2_cpu_utilization_24ae8d.csv
True Positive (Detected anomalies) : 402
True Negative (Detected non anomalies) : 0
False Positive (False alarms) : 3447
False Negative (Anomaly not detected) : 0
Total data points : 3428
S(t)_standard score : -94.2570109799537
